In [ ]:
#Set Up Modules:
#--------------------------------------------------------------------------------------
import numpy as np                 #library for working with arrays
import matplotlib.pyplot as plt    #libary for plotting (extension of numpy)
import re as regex                 #library for regular expressions
import cv2                         #libary to solve computer vision problems
import math                        #math tools
import random                      #library for randomization tools
import time                        #library for timing tools
import statistics as stat
import pickle

import winsound
import os, sys
from os import listdir, makedirs   #to use "listdir" and "mkdir"
from os.path import isfile, join, exists   #to use file tools
import copy

# from easy_table import EasyTable
import keras
from keras.models import model_from_json
from keras.backend import manual_variable_initialization 
manual_variable_initialization(True)
from PIL import Image


#Functions I wrote:
#-------------------------------------------------------------------
import CustomAssertions
import Distance
import Imaging
from Imaging import FrameManager, Background, CustomImage
import FeatureTools
import HandAssessments
from Analysis import StopWatch


In [ ]:
### SETUP ENVIRONMENT ###
#---search paths
rootFolder = r"C:\Users\james\Desktop\Work_Spaces\00-School\02-Graduate\07-AdvTopicDeepLearning\00-Drone_Project"
dataSubFolder = r"04-Data2\01-RawImages_All"
modelSubFolder = r"06-Models\00-Model_1"
# modelName = 'MODEL_1_RGB_Apr5_2022'
modelName = 'MODEL_1_Apr4_2022'
ResultsSubfolder = r"04-Data2\03-AdditionalBackground_100x100\01-Grayscale"
fileSearchRE = "[0-9]_Feet_[0-9_]+.jpg"

#---options
imgShrinkFactor = 0.30     #% (fraction) of current size (keep aspect ratio)
finalImageSize = (100,100) #% ((pixels,pixels)) size of training sample image
singleStrat = True
distance = [2,3,4]
color = "Grayscale"

#---load json strings into memory
json_file_gestures = open(rootFolder + '/' + modelSubFolder + '/' + modelName + '.json', 'r')
loaded_model_json_gestures = json_file_gestures.read()
json_file_gestures.close()
Model_1   = model_from_json(loaded_model_json_gestures)

#---load weights into new model
Model_1.load_weights(rootFolder + '/' + modelSubFolder + '/' + modelName + ".h5")



In [ ]:
def WriteImageToFile(pathToImageFolder, imageName, img):
    pathToImage = pathToImageFolder + '\\' + imageName + ".jpg"
    cv2.imwrite(pathToImage, img)

    
def eliminateInnerBoxes(boxes):
    boxes.sort(key=lambda x: ((x[0]+x[2]) * (x[1]+x[3])), reverse=True)
    print("sorted boxes: ", boxes)
    #cycle through boxes, pop boxes from list when they are found to be duplicates of the reference one.
    endPoint = len(boxes)-1 
    i = 0
    while i < endPoint:
        #for all other elements, left to right, calculate IOU
        copies = [j for j in range(i+1,endPoint+1) if isBoxInsideBox(boxes[j],boxes[i])]

        #remove all copies from list
        boxes = [val for index,val in enumerate(boxes) if index not in copies]

        #update item in list to use next
        endPoint -= len(copies)
        i += 1        
    return boxes
    
def isBoxInsideBox(box1,box2):
    return box1[0] >= box2[0] and box1[1] >= box2[1] and box1[0]+box1[2] <= box2[0]+box2[2] and box1[1]+box1[3] <= box2[1]+box2[3]




    
def NonMaximumSuppression(boxes, confidences, IOU_Allowance=0.40):
    boxConfidence = list(zip(boxes, confidences))
    boxConfidence.sort(key=lambda x: x[1], reverse=True)
    
    #cycle through boxes, pop boxes from list when they are found to be duplicates of the reference one.
    endPoint = len(boxConfidence)-1 
    i = 0
    while i < endPoint:
        #for all other elements, left to right, calculate IOU
        copies = [j for j in range(i+1,endPoint+1) if IOU(boxConfidence[i][0], boxConfidence[j][0]) > IOU_Allowance]

        #remove all copies from list
        boxConfidence = [val for index,val in enumerate(boxConfidence) if index not in copies]

        #update item in list to use next
        endPoint -= len(copies)
        i += 1      
    
    boxes = []
    for item in boxConfidence:
        boxes.append(item[0])
    return boxes


def IOU(a, b, epsilon=1e-5):
    """
        Intersection over union to identify the most unique region proposals so that there are not duplicate images.
    """
    #coordinates of intersection box
    x1 = max(a[0], b[0])
    y1 = max(a[1], b[1])
    x2 = min(a[0] + a[2], b[0] + b[2])
    y2 = min(a[1] + a[3], b[1] + b[3])

    #area where the boxes intersect
    area_overlap = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)

    #combined area
    area_a = (a[2] + 1) * (a[3] + 1)
    area_b = (b[2] + 1) * (b[3] + 1)
    area_combined = area_a + area_b - area_overlap
    return abs(area_overlap / (area_combined+epsilon))

In [ ]:
#Prepare background images
#----------------------------------------------------------------------------------------------------------------
    
#--------------------------------------------- AUTOMATION SECTION --------------------------------------------------
#color to image shape
if color == "RGB":
    imageLayers = 3
else:
    imageLayers = 1

#initialize overall timing stuff
SelectiveSearchModel = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

#initialize overall timing stuff
for dist in distance:
    #grab new frame manager
    frameManager = FrameManager("%d_Feet" % dist, rootFolder, dataSubFolder, ResultsSubfolder, fileRegEx=fileSearchRE, COLOR="RGB")
    if frameManager._managerStatus is -1:
        continue

    #run algorithm for one set of images 
    while(frameManager.imagesOnStack >=1): 
        #update counters for image labeling
        frameManager.setNextFrame()                             #get next frame on stack
        c_Frame = frameManager.getCurrentFrame()                #snag current frame
        print("----> Frame: ", frameManager.currentImageName)   #print name of frame
        img = c_Frame.image
        imgSize = np.shape(img)                       #capture image size (different sizes depending on cropping)
        img = cv2.resize(img, (int(imgShrinkFactor*imgSize[1]), int(imgShrinkFactor*imgSize[0])), interpolation=cv2.INTER_LANCZOS4)

        SelectiveSearchModel.setBaseImage(img)
        SelectiveSearchModel.switchToSelectiveSearchFast()
        if singleStrat == True: 
            SelectiveSearchModel.switchToSingleStrategy()   
        rects = SelectiveSearchModel.process()
        print("---->BOXES FOUND:", len(rects))

        #show sample images
        if color == 'Grayscale':
            test = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY)
        else:
            test = img.copy()
        listOfBoxBoxes = []
        predictionArray = []
        for (x, y, w, h) in rects:
            thisTest = test.copy()
            snippetSize = np.shape(test[y:y+h, x:x+w])
            if snippetSize[0]*snippetSize[1] > 0:
                rs_test = cv2.resize(test[y:y+h, x:x+w], finalImageSize, interpolation=cv2.INTER_LANCZOS4)
                rs_test = rs_test.astype('float32')
                rs_test /=  255.0
                prediction = Model_1(np.reshape(rs_test,(-1,100,100,imageLayers)), training=False).numpy()
                predictedClass = "BOX" if prediction[0][1] > 0.85 else "NOTBOX"
                if predictedClass == "BOX":
                    listOfBoxBoxes.append([x, y, w, h])
                    predictionArray.append(prediction[0][1])
        
#         for (x, y, w, h) in listOfBoxBoxes:
#             cv2.putText(thisTest, "[BOX]", (x,y+h), cv2.FONT_HERSHEY_SIMPLEX, 0.4, 255)
#             cv2.rectangle(thisTest, (x, y), (x + w, y + h), [90,90,90], 2)
#         print("BEFORE: ", listOfBoxBoxes)
#         Imaging.showImage(thisTest)
        
        nonDuplicates = NonMaximumSuppression(listOfBoxBoxes, predictionArray, IOU_Allowance=0.10)
        removedInnerBoxes = eliminateInnerBoxes(nonDuplicates)
        thisTest = test.copy()
        for (x, y, w, h) in removedInnerBoxes:
            cv2.putText(thisTest, "[BOX]", (x,y+h), cv2.FONT_HERSHEY_SIMPLEX, 0.4, 255)
            cv2.rectangle(thisTest, (x, y), (x + w, y + h), [90,90,90], 2)
#         print("AFTER: ", reducedBoxes)
        Imaging.showImage(thisTest)        
        
        

                


Number of images found -->  633
----> Frame:  2_Feet_1.jpg
---->BOXES FOUND: 79
sorted boxes:  [[0, 0, 262, 59]]
----> Frame:  2_Feet_10.jpg
---->BOXES FOUND: 95
sorted boxes:  [[7, 0, 281, 216], [174, 158, 51, 58], [206, 113, 24, 45], [206, 39, 34, 38]]
----> Frame:  2_Feet_100.jpg
---->BOXES FOUND: 168
sorted boxes:  [[51, 35, 148, 181]]
----> Frame:  2_Feet_101.jpg
---->BOXES FOUND: 160
sorted boxes:  [[48, 34, 149, 182], [67, 162, 35, 54]]
----> Frame:  2_Feet_102.jpg
---->BOXES FOUND: 198
sorted boxes:  [[43, 55, 151, 161], [57, 179, 62, 37], [53, 144, 48, 38], [74, 55, 50, 31]]
----> Frame:  2_Feet_103.jpg
---->BOXES FOUND: 178
sorted boxes:  [[61, 50, 128, 166], [87, 0, 35, 82]]
----> Frame:  2_Feet_104.jpg
---->BOXES FOUND: 165
sorted boxes:  [[0, 82, 168, 134], [147, 107, 19, 46], [67, 82, 44, 47]]
----> Frame:  2_Feet_105.jpg
---->BOXES FOUND: 181
sorted boxes:  [[0, 98, 167, 118], [106, 48, 20, 41]]
----> Frame:  2_Feet_106.jpg
---->BOXES FOUND: 185
sorted boxes:  [[72, 65, 

KeyboardInterrupt: 

In [ ]:
print(IOU([0,0,2,4], [0,0,4,4]))
print(IOU([0,0,10,10], [0,0,2,4]))
print(NonMaximumSuppression([[0, 132, 144, 2], [0, 132, 149, 5], [0, 132, 144, 5]], predictionArray))


0.599999760000096
0.12396693190355934
[0.483333327962963, 0.966666655925926]
[[0, 132, 149, 5]]


In [ ]:
def NonMaximumSuppression(boxes, confidences, IOU_Allowance=0.40):
    boxConfidence = list(zip(boxes, confidences))
    boxConfidence.sort(key=lambda x: x[1], reverse=True)
    print(boxConfidence)
    
    #cycle through boxes, pop boxes from list when they are found to be duplicates of the reference one.
    endPoint = len(boxConfidence)-1 
    i = 0
    while i < endPoint:
        print("boxConfidence_1:", boxConfidence[0])
        print("boxConfidence_1:", boxConfidence[1])
        #for all other elements, left to right, calculate IOU
        copies = [j for j in range(i+1,endPoint+1) if IOU(boxConfidence[i][0], boxConfidence[j][0]) > IOU_Allowance]
        print([IOU(boxConfidence[i][0], boxConfidence[j][0]) for j in range(i+1,endPoint+1)])
        #remove all copies from list
        boxConfidence = [val for index,val in enumerate(boxConfidence) if index not in copies]

        #update item in list to use next
        endPoint -= len(copies)
        i += 1      
    return boxes    
    
    
confidence = predictionArray
boxes = listOfBoxBoxes
print(NonMaximumSuppression([[0, 132, 144, 2], [0, 132, 149, 5], [0, 132, 144, 5]], confidence))


[([14, 69, 84, 102], 0.999813), ([14, 67, 68, 101], 0.98575634)]
boxConfidence_1: ([14, 69, 84, 102], 0.999813)
boxConfidence_1: ([14, 67, 68, 101], 0.98575634)
[0.7758911494704924]
[[14, 69, 84, 102], [14, 67, 68, 101]]
0.7758911494704924


In [ ]:
def eliminateInnerBoxes(boxes):
    boxes.sort(key=lambda x: ((x[0]+x[2]) * (x[1]+x[3])), reverse=True)
    print("sorted boxes: ", boxes)
    #cycle through boxes, pop boxes from list when they are found to be duplicates of the reference one.
    endPoint = len(boxes)-1 
    i = 0
    while i < endPoint:
        #for all other elements, left to right, calculate IOU
        copies = [j for j in range(i+1,endPoint+1) if isBoxInsideBox(boxes[j],boxes[i])]

        #remove all copies from list
        boxes = [val for index,val in enumerate(boxes) if index not in copies]

        #update item in list to use next
        endPoint -= len(copies)
        i += 1        
    return boxes
    
def isBoxInsideBox(box1,box2):
    return box1[0] >= box2[0] and box1[1] >= box2[1] and box1[0]+box1[2] <= box2[0]+box2[2] and box1[1]+box1[3] <= box2[1]+box2[3]

    
    
    
boxList = [ [0,0,10,10],
            [1,1,2,2],
            [1,1,6,6]]
print(isBoxInsideBox(boxList[1],boxList[2]))
print(eliminateInnerBoxes(boxList))

True
sorted boxes:  [[0, 0, 10, 10], [1, 1, 6, 6], [1, 1, 2, 2]]
[[0, 0, 10, 10]]
